In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

# Set up: Create session and People table with the People model.
class People(Base):
    """Characters mentioned and their role in the puzzle."""

    __tablename__ = "people"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False, unique=True)
    role = Column(String, nullable=True)

engine = create_engine("sqlite:///temp_test.db", echo=True)
conn = engine.connect()
session = sessionmaker(bind=engine)()
Base.metadata.create_all(engine)




In [ ]:
# Add records to the People table
c1 = People(id=1, name="Sprinkles", role="Test")
c2 = People(id=2, name="Sparkles", role="Test")
session.add(c1)
session.add(c2)
session.commit()

In [ ]:
# Create some data to query for
data = {"id": 2, "name": "Sparkles", "role": "Test"}

In [ ]:
# Generic functionality that uses the incoming data to form sqlalchemy queries 
model = People
filters = []
# Iterate over the dictionary and for each key, check if the model has a column of the same name
for k, v in data.items():
    # Get the column from the model or None
    column = getattr(model, k, None)
    # If the model has a matching column and the value is truthy, then append the column == value statement to the query filters
    # This will add 3 statements to the filters list, which will be joined with AND
    if column and v:
        filters.append((getattr(column, "__eq__")(v)))

In [ ]:
# Check the attributes of the filters
[x.__dict__ for x in filters]

In [ ]:
# Query using the filters list built by the for loop
result = session.query(model).filter(*filters).one_or_none()

In [ ]:
# Check that it returned the correct record
result.name

In [ ]:
# Clean up: remove db file
from pathlib import Path

path = Path("temp_test.db")

if path.exists():
    path.unlink()
